# Problem Set 2

CBE 60258, University of Notre Dame. © Prof. Alexander Dowling, 2023

You may not distribution homework solutions without written permissions from Prof. Alexander Dowling.


**Your Name and Email:**

## Import Libraries, Define Necessary Functions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
'''
This cell installs Pyomo and Ipopt on Google Colab. To run this notebook
locally (e.g., with anaconda), you first need to install Pyomo and Ipopt.
'''

import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/IDAES/idaes-pse/main/scripts/colab_helper.py"
    import colab_helper
    colab_helper.install_idaes()
    colab_helper.install_ipopt()

In [ ]:
def Jacobian(f,x,delta = 1.0e-7):
    '''Approximate Jacobian using forward finite difference

    Args:
        f: vector-valued function
        x: point to build approximation J(x) around
        delta: finite difference step size

    Returns:
        J: square Jacobian matrix (approximation)

    '''
    # Determine size
    N = x.size
    
    #Evaluate function f at x
    fx = f(x) #only need to evaluate this once
    
    # Make sure f is square (no. inputs = no. outputs)
    try:
        assert N == fx.size, "Your problem is not square!"
    except AssertionError:
        print("x = ",x)
        print("fx = ",fx)
    
    
    # Allocate empty matrix
    J = np.zeros((N,N))

    idelta = 1.0/delta #division is slower than multiplication
    x_perturbed = x.copy() #copy x to add delta
    
    # Loop over elements of x and columns of J
    for i in range(N):
        # Perturb (apply step) to element i of x
        x_perturbed[i] += delta
        
        # Approximate column in Jacobian
        col = (f(x_perturbed) - fx) * idelta
        
        # Reset element of x
        x_perturbed[i] = x[i]
        
        # Save results
        J[:,i] = col
    # end for loop
    return J

def newton_system(f,x0,exact_Jac=None,delta=1E-7,epsilon=1.0e-6, LOUD=False):
    """Find the root of the function f via exact or inexact Newton-Raphson method
    Args:
        f: function to find root of
        x0: initial guess
        exact_Jac: function to calculate J. If None, use finite difference
        delta: finite difference tolerance. Only used if J is not specified
        epsilon: tolerance
        
    Returns:
        estimate of root
    """
        
    x = x0
    if (LOUD):
        print("x0 =",x0)
    iterations = 0
    fx = f(x)
    while (np.linalg.norm(fx) > epsilon):
        if exact_Jac is None:
            # Use finite difference
            J = Jacobian(f,x,delta)
        else:
            J = exact_Jac(x)
        
        RHS = -fx;
        
        # solve linear system
        # We could have used GaussElimPivotSolve here instead
        delta_x = np.linalg.solve(J,RHS)
        
        # Check if GE returned any NaN values
        if np.isnan(delta_x).any():
            print("Gaussian Elimination Failed!")
            print("J = \n",J,"\n")
            print("J is rank",np.linalg.matrix_rank(J),"\n")
            print("RHS = ",RHS)
        x = x + delta_x
        fx = f(x)
        if (LOUD):
            print("\nIteration",iterations+1,": x =",x,"\n norm(f(x)) =",np.linalg.norm(fx),"\t cond(J(x)) =",np.linalg.cond(J))
        iterations += 1
    print("\nIt took",iterations,"iterations")
    return x #return estimate of root

## 1. Solubility of Carbon Dioxide in Aqueous NaOH

The solubility of $\mathrm{CO_2}$ in water is a strong function of the pH, due to the conversion of dissolved $\mathrm{CO_2}$ to carbonic acid ($\mathrm{H_2CO_3}$) and its dissociation into $\mathrm{HCO_3^-}$ and $\mathrm{CO_3^{2-}}$.

In this problem, you will predict the equilibrium concentration of all dissolved species in a 0.01 M NaOH solution. Assuming that the NaOH completely dissociates (it is a very strong base), we want to calculate the pH and species concentrations for a solution in equilibrium with 0.2 atm $\mathrm{CO_2}$ gas (i.e. $\mathrm{P_{CO_2}}$ = 0.2 atm). 
*This would roughly be the concentration of $\mathrm{CO_2}$ in a stoichiometric burning of coal in air with all oxygen consumed. This chemistry is important for carbon capture from a power plant with a base solution.*

We have the following equilibrium relationships:

$$\frac{\lbrack CO_2 \rbrack}{P_{CO_2} } = K_{abs}$$

$$\frac{\lbrack H_2CO_3 \rbrack}{\lbrack CO_2 \rbrack } = K_{eq}$$

$$\frac{\lbrack HCO_3^- \rbrack \times \lbrack H^+ \rbrack}{\lbrack H_2CO_3 \rbrack } = K_{a_1}$$

$$\frac{\lbrack CO_3^{2-} \rbrack \times \lbrack H^+ \rbrack}{\lbrack HCO_3^- \rbrack } = K_{a_2}$$

$$\lbrack H^+ \rbrack \times \lbrack OH^- \rbrack = K_{w}$$

The equilibrium constants have the following values:

| Constant |     Value      | Units |
|-|-|-|
|$K_{abs}$ | $3.36 \times 10^{-2}$ | M/atm |
|$K_{eq}$ | $1.7 \times 10^{-3}$ | - |
|$K_{a_1}$ | $2.5 \times 10^{-4}$ | M |
|$K_{a_2}$ | $4.8 \times 10^{-11}$ | M |
|$K_{w}$ | $1.00 \times 10^{-14}$ | M$^2$ |

Further, the electroneutrality condition for the system is given by the equation:

\begin{equation}
\lbrack Na^+ \rbrack + \lbrack H^+ \rbrack 
= \lbrack OH^- \rbrack + \lbrack HCO_3^- \rbrack + 2\lbrack CO_3^{2-} \rbrack 
\end{equation}



### 1a. Naïve Model

Write down the system of nonlinear equations in canonical form to predict the equilibrium concentrations. Clearly define all the variables that you use. In *2c* you will refine the model to improve condition, so *do not* transform the the variable in this part.

*Hint*: Start by performing a degree of freedom analysis.



### 1b. Numeric Solution with Python

Numerically solve your model from *2a*. We will see in this problem there are many different answers depending on the order of equations and the initial point. This is because the problem is nonlinear and exacerbated by the fact our naïve model is poorly scaled. 

In [ ]:
# lists of species 
species = ["H", "HCO3", "CO3"]
other = ["CO2", "H2CO3", "OH"]

# Provided specification
K_abs = 3.36e-2 # M/atm
K_eq = 1.7e-3
K_a1 = 2.5e-4 # M
K_a2 = 4.8e-11 #M
K_w = 1e-14 # M^2

# CO2 pressure
P_co2 = 0.2 # atm

# Molarity of solution is Na+ concentration
Na_con = 0.01 # M

# define the set of equations
def my_equations_1(x):
    ''' Nonlinear system of equations in conancial form c(x) = 0
    
    Arg:
        x: vector of variables
        
    Returns:
        r: residual, 
    
    '''
    # allocate an empty array
    r = np.empty((3))
  
    # equations
    
    # Add your solution here
  
    return r

Specify an initial point and solve your model in Python.

In [ ]:
# Add your solution here

Specify a different initial point and solve your model in Python. Hint: You might get a different answer with a different initial point.

In [ ]:
# Add your solution here

### 1c. Refined Model

Reformulate your model from **a** to a model that is better conditioned.

### 1d. Numeric Solution with Python

Numerically solve your model from **c**. First, define the refined model in canonical form.

In [ ]:
# lists of species
species = ["H", "HCO3", "CO3"]
other = ["CO2", "H2CO3", "OH"]

# Provided specification
K_abs = 3.36e-2 # M/atm
K_eq = 1.7e-3
K_a1 = 2.5e-4 # M
K_a2 = 4.8e-11 #M
K_w = 1e-14 # M^2

# CO2 pressure
P_co2 = 0.2 # atm

# Molarity of solution is Na+ concentration
Na_con = 0.01 # M

# define the set of equations
def my_equations2(x):
    ''' Nonlinear system of equations in conancial form c(x) = 0
    Arg:
        x: vector of variables
    Returns:
        r: residual
    '''
    # allocate an empty array
    r = np.empty((3))
    
    # equations
    
    # Add your solution here
    
    return r



Next, specify one initial point and solve the model.

In [ ]:
# Add your solution here

Now, try another initial point.

In [ ]:
# Add your solution here

Record your answers for this problem in the template provided in Canvas and turn in via **Gradescope.**

### 1e. Discussion

Why is transforming the equations so important for this problem? Write 2 to 4 sentences. For full credit, please discuss i) condition number, ii) scaling, and iii) convergence tolerance for Newton's method.

**Your Answer**:

## 2. Numeric Integration: Reaction Kinetics

The species A undergoes a dimerization reaction:

\begin{equation}
2A\rightarrow B
\end{equation}

With the rate:

\begin{equation}
r_1 = k_1 \cdot C_A^2
\end{equation} 

The reaction continues, however, with the further reaction:

\begin{equation}
B+A\rightarrow C
\end{equation} 

With the rate:

\begin{equation}
r_2 = k_2 \cdot C_A \cdot C_B
\end{equation}

We would like to maximize the product B, so we need to stop the reaction when this is at a maximum.  We will consider $k_1=2$ and $k_2=4$ L/(mol-s) with the initial conditions $C_A=1$ and $C_B=0$ mol/L.

As seniors, you will learn how to apply a mass balance to convert reaction rate expressions into differential equations:

\begin{equation}
	\frac{dC_A}{dt}=-2\cdot r_1 - r_2 = -2\cdot k_1 \cdot C_A^2-k_2 \cdot C_A \cdot C_B
\end{equation}

\begin{equation}
	\frac{dC_B}{dt}=r_1 - r_2 = k_1 \cdot C_A^2-k_2 \cdot C_A \cdot C_B
\end{equation}

Do the following:
1. Calculate the concentrations $C_A$ and $C_B$ as a function of time.
2. Plot the concentrations from $t=0$ to $t=3$.
3. Estimate the maximum concentration of B and when it occurs. Mark this point on your plot. Save this value in the float `maxCB`.



Hint: Because this is not system of linear differential equations, we should use `scipy.integrate.solve_ivp`. Check your notes from Class 12 and/or explore the Scipy documentation for examples: https://docs.scipy.org/doc/scipy-1.1.0/reference/generated/scipy.integrate.solve_ivp.html

In [ ]:
#Solve ODES
def f(t, y):
    ''' RHS of differential equation for reaction kinetics
    Args:
        t: time
        y: values for differential questions, [CA, CB]
    Returns:
        dydt: first derivation of y w.r.t. t
    
    '''
    # unpack current values of y
    CA, CB = y      
    
    # define kinetic parameters
    k1 = 2
    k2 = 4
    
    #Store equations as a list in the variable dydt
    
    # Add your solution here
    
    return dydt

# Initial values
CA0 = 1.0     
CB0 = 0.0 

# Bundle initial conditions for ODE solver
y0 = [CA0, CB0]

t = np.arange(0, 3, 0.01)
tspan = [np.min(t), np.max(t)]

# Call the ODE solver
# Note: The solution profile changes slightly if we change the integration method.
soln = integrate.solve_ivp(f, tspan, y0, t_eval=t, method='RK23')

# You may wish to uncomment the following line to take a closer look.
# print(soln)

CA = soln.y[0]
CB = soln.y[1]

#Find maximum concentration of B
# Add your solution here

print("maxCB =",maxCB)

#Plot
# Add your solution here

In [ ]:
# Removed autograder test. You may delete this cell.

## 3. Numeric Integration: Cannonball

An iron cannonball of mass 1kg is fired upwards at a velocity of 200m/s.  In addition to gravity, there is a drag due to air resistance given by $F_{D} = -0.00292 U|U|$ where $U$ is $dh/dt$ (all units SI).  Note that we have to use $U|U|$ rather than $U^2$ because we have to keep track of the sign.

Do the following:
1. Use Newton's laws to construct a pair of coupled first order differential equations to track the velocity and height of the cannon ball. Write this system on paper. Turn this in via **Gradescope**.
2. Integrate the model in Python. *Hint*: How should you integrate this numerically?
3. Plot the height and velocity as a function of time. Determine the maximum elevation of the cannonball and mark it on the graph. Save this value in the float `maxh`. Then print the value.

For a token amount of extra credit, prepare your plot with velocity on the left vertical axis and elevation on the right vertical axis. See this example for starter code: https://matplotlib.org/gallery/api/two_scales.html

In [ ]:
# Add your solution here

In [ ]:
# Removed autograder test. You may delete this cell.

## 4. Portfolio Data Analysis

Portfolio management is a classic example of quadratic programming (optimization). The idea is to find the optimal blend of investments that achieves a specified rate of return (or better) while minimizing the variance in rate of return. In this problem, you will use your skills in statistical analysis to analyze the stock data.

### Historical Stock Data

Historical daily adjusted closing prices for the last five years (obtained from Yahoo! Finance) are available for the $N=5$ stocks listed in table below. (We are actually considering index funds, but this detail does not change the analysis.)  

| Symbol | Name |
|-|-|
| GSPC | S&P 500 | 
| DJI | Dow Jones Industrial Average | 
| IXIC | NASDAQ Composite | 
| RUT | Russell 2000 |
| VIX | CBOE Volatility Index |

### 4a. Return Rate

You are given a Stock\_Data.csv file. Using the stock data, calculate the 1-day return rate:

\begin{equation}
	r_{t,i} = \frac{p_{t+1,i} - p_{t,i}}{p_{t,i}}
\end{equation}

where $p_{t+1,i}$ and $p_{t,i}$ are the *Adjusted Closing Prices* at the end of days $t+1$ and $t$, respectively, for stock $i$. These results are stored in matrix `R`. *Hint*: Use Pandas.

In [ ]:
# This is the long path to the folder containg data files on GitHub (for the class website)
data_folder = 'https://raw.githubusercontent.com/ndcbe/data-and-computing/main/noteboohttps://raw.githubusercontent.com/ndcbe/data-and-computing/main/notebooks/data/'

# Load the data file into Pandas
df_adj_close = pd.read_csv(data_folder + 'Stock_Data.csv')

In [ ]:
# Add your solution here

### 4b. Visualization

Plot the single day return rates for the 5 stocks you obtain in the previous section and check if you obtain the following profiles:

![ad](https://raw.githubusercontent.com/ndcbe/data-and-computing/main/media/stock_return_plots.png)


The first plot is made for you. 

In [ ]:
# Create figure
plt.figure(figsize=(9,15))

# Create subplot for DJI
plt.subplot(5,1,1)
plt.plot(R["DJI"]*100,color="blue",label="DJI")
plt.legend(loc='best')

# Add your solution here

# Show plot
plt.show()

### 4c. Covariance and Correlation Matrices

Write Python code to:
1. Calculate $\bar{r}$, the average 1-day return for each stock. Store this as the variable `R_avg`.
2. Calculate $\Sigma_{r}$, the covariance matrix of the 1-day returns. This matrix tells us how returns for each stock vary with each other (which is important because they are correlated!). Hint: pandas has a function `cov`
3. Calculate the correlation matrix for the 1-day returns. Hint: pandas has a function `corr`.

Looking at the correlation matrix, answer the follwing questions:

1. Which pair of stocks have the highest **positive** correlation?
2. Which pair of stocks have the highest **negative** correlation?
3. Which pair of stocks have the lowest **absolute** correlation?

Hint: Read ahead in the class website for more information on [correlation and covariance](../..//notebooks/14/Correlation-Covariance-and-Independence.ipynb)

Please write one or two sentences for each question:

In [ ]:
# Add your solution here

In [ ]:
# Removed autograder test. You may delete this cell.

### 4c. Markowitz Mean/Variance Portfolio Model

The Markowitz mean/variance model, shown below, computes the optimal allocation of funds in a portfolio:

\begin{align}
		\min_{{x} \geq {0}} \qquad & z:= {x}^T \cdot {\Sigma_r} \cdot {x} \\
		\text{s.t.} \qquad & {\bar{r}}^T \cdot {x} \geq \rho \\
		 & \sum_{i =1}^N x_i = 1 
\end{align} 


where $x_i$ is the fraction of funds invested in stock $i$ and $x = [x_1, x_2, ..., x_N]^T$. The objective is to minimize the variance of the return rate. (As practice for the next exam, try deriving this from the error propagation formulas.) This requires the expected return rate to be at least $\rho$. Finally, the allocation of funds must sum to 100%.

Write Python code to solve for $\rho = 0.08\%.$ Report both the optimal allocation of funds and the standard deviation of the return rate. 
*Hint*:
1. Be mindful of units.
2. You can solve with problem using the Pyomo function given below
3. $:=$ means ''defined as''

Store your answer in `std_dev`.

In [ ]:
R_avg_tolist = R_avg.values.tolist()
Cov_list = Cov.values.tolist()

# Optimization Problem
def create_model(rho,R_avg,Cov):
    
    '''
    This function solves for the optimal allocation of funds in a portfolio 
    by minimizing the variance of the return rate
    
    Arguments:
        rho: required portfolio expected return
        Ravg: average return rates (list)
        Cov: covariance matrix
        
    Returns:
        m: Pyomo concrete model
    
    '''
    
    m = pyo.ConcreteModel()
    init_x = {}
    m.idx = pyo.Set(initialize=[0,1,2,3,4])
    for i in m.idx:
        init_x[i] = 0
    m.x = pyo.Var(m.idx,initialize=init_x,bounds=(0,None))
    
    def Obj_func(m):
        b = []
        mult_result = 0
        for i in m.idx:
            a = 0
            for j in m.idx:
                a+= m.x[j]*Cov[j][i]
            b.append(a)
        for i in m.idx:
            mult_result += b[i]*m.x[i]
        
        return mult_result
    m.OBJ = pyo.Objective(rule=Obj_func)
    
    def constraint1(m):
        # Add your solution here

    m.C1 = pyo.Constraint(rule=constraint1)
    
    def constraint2(m):
        # Add your solution here

    m.C2 = pyo.Constraint(rule=constraint2)
    
    return m


In [ ]:
rho = 0.0008
model1 = create_model(rho,R_avg_tolist,Cov_list)

#Solve Pyomo in the method learned in Class 11

# Add your solution here

In [ ]:
# Removed autograder test. You may delete this cell.

### 4e. Volatility and Expected Return Tradeoff

We will now perform sensitivity analysis of the optimization problem in 3d to characterize the tradeoff between return and risk.

Write Python code to:
1. Solve the optimization problem for many values of $\rho$ between min($\bar{r}$) and max($\bar{r}$) and save the results. Use the Pyomo function created in 3d.
2. Plot $\rho$ versus $\sqrt{z}$ (using the saved results).
3. Write at least one sentence to interpret and discuss your plot.

Submit your plot and discussion via **Gradescope**.

In [ ]:
rho_vals = np.arange(0.0005,0.0038,0.0001)
std_dev = []

# Add your solution here

In [ ]:
#Plot

# Add your solution here

**Discussion**: